In [ ]:
filters_path = '../metadata/dsets.csv'
dataset_str = 'Laughney_Massague_2020_NSCLC'
adatas_path = '/root/datos/maestria/netopaas/luca/data/'


data_dir = '/root/datos/maestria/netopaas/lung_scRNA'
backup_dir = '/root/datos/maestria/netopaas/luca_explore'

In [7]:
import pandas as pd
import scanpy as sc

In [14]:
filters = pd.read_csv(filters_path)
id_row = filters[filters.id == dataset_str]
file_str = '/'.join(id_row.input_adata.iloc[0].split('/')[1:])

thresholds = {}
thresholds['min_counts']  = int(id_row.min_counts)
thresholds['max_counts'] = int(id_row.max_counts)
thresholds['min_genes'] = int(id_row.min_genes)
thresholds['max_genes'] = int(id_row.max_genes)
thresholds['max_pct_mito'] = int(id_row.max_pct_mito)

adata = ad.read_h5ad(f'{adatas_path}/{file_str}')

/tmp/ipykernel_52065/2622408628.py:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  thresholds['min_counts']  = int(id_row.min_counts)
/tmp/ipykernel_52065/2622408628.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  thresholds['max_counts'] = int(id_row.max_counts)
/tmp/ipykernel_52065/2622408628.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  thresholds['min_genes'] = int(id_row.min_genes)
/tmp/ipykernel_52065/2622408628.py:5: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  thresholds['max_genes'] = int(id_row.max_genes)
/tmp/ipykernel_52065/2622408628.py:6: FutureWarning: Calling int on a single element Se

AnnData object with n_obs × n_vars = 42847 × 23278
    obs: 'sample', 'patient', 'origin', 'condition', 'tissue'

In [ ]:
if dataset_str == 'Zilionis_Klein_2019_NSCLC':
    adata = adata[
        adata.obs["tissue"] == "lung", :
    ]

In [2]:
import seaborn as sns

In [ ]:
p1 = sns.displot(adata.obs["total_counts"], bins=100, kde=False)


In [12]:
#papermill_description=FILTERS

# very basic gene filtering - genes with 0 cells cause some downstream processes to fail.
print("Filtering genes")
print(f"    Before: {adata.shape[1]}")
sc.pp.filter_genes(adata, min_counts=2)
print(f"    After: {adata.shape[1]}")

# %%
# Apply thresholds
print("Filter by min_counts")
print(f"    Before: {adata.shape[0]}")
sc.pp.filter_cells(adata, min_counts=thresholds["min_counts"])
print(f"    After: {adata.shape[0]}")


print("Filter by max_counts")
print(f"    Before: {adata.shape[0]}")
sc.pp.filter_cells(adata, max_counts=thresholds["max_counts"])
print(f"    After: {adata.shape[0]}")


print("Filter by min_genes")
print(f"    Before: {adata.shape[0]}")
sc.pp.filter_cells(adata, min_genes=thresholds["min_genes"])
print(f"    After: {adata.shape[0]}")


print("Filter by max_genes")
print(f"    Before: {adata.shape[0]}")
sc.pp.filter_cells(adata, max_genes=thresholds["max_genes"])
print(f"    After: {adata.shape[0]}")

# %%
if "mito" not in adata.var.columns:
    adata.var["mito"] = adata.var_names.str.lower().str.startswith("mt-")

# %%
sc.pp.calculate_qc_metrics(
    adata, qc_vars=("mito",), log1p=False, inplace=True, percent_top=None
)

print("Filter by max_pct_mito")
print(f"    Before: {adata.shape[0]}")
adata = adata[adata.obs["pct_counts_mito"] < thresholds["max_pct_mito"]].copy()
print(f"    After: {adata.shape[0]}")

Filtering genes
    Before: 23278
    After: 21595
Filter by min_counts
    Before: 42847
    After: 30303
Filter by max_counts
    Before: 30303
    After: 29710
Filter by min_genes
    Before: 29710
    After: 29690
Filter by max_genes
    Before: 29690
    After: 29690
Filter by max_pct_mito
    Before: 29690
    After: 29690


In [ ]:
adata.write_h5ad(f'{backup_dir}/surgeries/filtered_{dataset_str}.h5ad')